# Jupyter Widgets

## Notebook to PDF Converter

用 Javascript ```window.location.href``` 可以知道當前 server url。下面這個 download button 的 callback 還有 bug。不過 [Volia 一遇到 Javascript 會有很多問題](https://github.com/voila-dashboards/voila/issues/108)，就算把下面這個修好也不一定能用 Voila 佈署

In [10]:
from ipywidgets import FileUpload, Button, Output
from IPython.display import display, FileLink, HTML, Javascript
import subprocess, os

env = 'lab'   # change to 'voila' before deploying

output = Output()
fileInput = FileUpload()
convertButton = Button(description='Convert')

def convert(b):
    with output:
        print('Your PDF will be ready in a few seconds. ')
        nbName = list(fileInput.value.keys())[0]
        pdfName = nbName.replace('ipynb', 'pdf')
        with open(nbName, 'w+b') as f:
            f.write(fileInput.data[0]) 
        proc = subprocess.run(['jupyter', 'nbconvert', nbName, '--to', 'pdf'])
        if proc.returncode == 1:
            print('LaTeX compiling error!')
        else:
            pdfPath = '/'.join(['tree'] + str(FileLink('__dash1.pdf')).split('/')[3:])
            display(HTML('''
                        <html>
                        <body>
                        <script type="text/javascript">
                            function setHref() {
                                var server = window.location.href;
                                var homeIdx = server.indexOf("%s");
                                server = server.slice(0, homeIdx);
                                var href = server + "%s";
                                document.buttonForm.link.href = href;
                            }
                        </script>
                        <form name="buttonForm">
                            <a name="link" href="https://www.google.com" download>
                                <button class="p-Widget jupyter-widgets jupyter-button widget-button" onClick="setHref();return true;">
                                    Download PDF
                                </button>
                            </a>
                        </form>
                        </body>
                        </html>
                         '''% (env, pdfPath)))

convertButton.on_click(convert)

display(fileInput, convertButton, output) 

FileUpload(value={}, description='Upload')

Button(description='Convert', style=ButtonStyle())

Output()

沒有 callback 的按紐是可以用的

In [2]:
display(HTML('''
    <a name="link" href="https://www.google.com" download>
        <button class="p-Widget jupyter-widgets jupyter-button widget-button" onClick="setHref();return true;">
            Go to Google!
        </button>
    </a>
'''))

## Get Server URL and Pass to Python

下面這段 code 來自[這裡](https://stackoverflow.com/questions/31818127/can-a-jupyter-ipython-notebook-take-arguments-in-the-url/37134476#37134476)。可能在 Jupyter Notebook 可以跑可是在 JupyterLab 不行因為 JavaScript 被 block 了

In [4]:
%%javascript

IPython.notebook.kernel.execute("url = '" + window.location + "'")

<IPython.core.display.Javascript object>

## [Create New Cell Programmatically](https://discourse.jupyter.org/t/how-to-programmatically-add-serveral-new-cells-in-a-notebook-in-jupyterlab/4323)

In [3]:
def create_new_cell(contents):
    from IPython.core.getipython import get_ipython
    shell = get_ipython()
    payload = dict(
        source='set_next_input',
        text=contents,
        replace=False,
    )
    shell.payload_manager.write_payload(payload, single = False)

contents = 'print("Hello World")'
create_new_cell(contents=contents)

In [ ]:
print("Hello World")